# Preparation of the Geometric Polygons for Postal Code Visualization

#### Notebook Purpose:
The purpose of this notebook is to prepare the polygons of PLZ2 and PLZ5 level.
The datasets are saved as BigQuery Tables with the polygons transformed to the: geography data type.




### Loading Data

In [ ]:
# authenticate user

from google.colab import auth

auth.authenticate_user()

In [ ]:
# set parameters to use BigQuery Functionality

PROJECT_ID = 'solarinsight-383513' #@param {type: "string"}
!gcloud config set project {PROJECT_ID}

In [ ]:
import google.cloud.bigquery as bq

client = bq.Client(project=PROJECT_ID)

In [ ]:
# copy the file from the bucket/folder_name/file to our colab notebook

!gsutil cp gs://bucket-quickstart-solarinsight/geo_data/plz-5stellig.geojson .
!gsutil cp gs://bucket-quickstart-solarinsight/geo_data/plz-2stellig.geojson .


Copying gs://bucket-quickstart-solarinsight/geo_data/plz-5stellig.geojson...
\ [1 files][ 21.2 MiB/ 21.2 MiB]                                                
Operation completed over 1 objects/21.2 MiB.                                     
Copying gs://bucket-quickstart-solarinsight/geo_data/plz-2stellig.geojson...
\ [1 files][  7.2 MiB/  7.2 MiB]                                                
Operation completed over 1 objects/7.2 MiB.                                      


In [ ]:
# install geopandas used to read the geojson files

!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 80.8 MB/s eta 0:00:00


In [ ]:
# read the geojson files

import geopandas as gpd
geo_data_plz5 = gpd.read_file('plz-5stellig.geojson')
geo_data_plz2 = gpd.read_file('plz-2stellig.geojson')


### Data Preprocessing

In [ ]:
# extract the PLZ2 from PLZ5

geo_data_plz5['plz2'] = geo_data_plz5['plz'].apply(lambda x: x[:2])
geo_data_plz5 = geo_data_plz5.rename(columns = {'geometry':'geometry5', 'plz':'plz5'})

geo_data_plz5.head()

,plz5,note,qkm,einwohner,geometry5,plz2
0,01067,01067 Dresden,6.866862,11957,"POLYGON ((13.68689 51.06395, 13.68698 51.06344...",01
1,01069,01069 Dresden,5.351833,25491,"MULTIPOLYGON (((13.72009 51.04664, 13.72162 51...",01
2,01097,01097 Dresden,3.297752,14821,"POLYGON ((13.72548 51.06860, 13.73000 51.06477...",01
3,01099,01099 Dresden,58.506789,28018,"POLYGON ((13.74218 51.08979, 13.74199 51.08963...",01
4,01108,01108 Dresden,16.447222,5876,"POLYGON ((13.76543 51.17491, 13.76282 51.17392...",01


In [ ]:
geo_data_plz2 = geo_data_plz2.rename(columns = {'geometry':'geometry2', 'plz':'plz2'})
geo_data_plz2.head()

,plz2,qkm,einwohner,geometry2
0,01,5097.98,1227440,"POLYGON ((13.63115 51.50731, 13.63050 51.50712..."
1,02,3477.04,445361,"POLYGON ((14.13494 51.54129, 14.13149 51.53860..."
2,03,3113.65,304953,"POLYGON ((13.54857 51.74694, 13.54681 51.74622..."
3,04,6206.45,1200630,"MULTIPOLYGON (((12.17924 51.38758, 12.17930 51..."
4,06,9375.81,1336123,"MULTIPOLYGON (((10.89703 51.61191, 10.90103 51..."


### BigQuery Table Creation

In [ ]:
# using the pandas google big query library we can save the dataframe directly as table and replace if it already exists

import pandas_gbq

table_name = 'geo_data.plz2_polygone'
pandas_gbq.to_gbq(geo_data_plz2, table_name, project_id=PROJECT_ID, if_exists='replace')

In [ ]:
table_name = 'geo_data.plz5_polygone'
pandas_gbq.to_gbq(geo_data_plz5, table_name, project_id=PROJECT_ID, if_exists='replace')

### Bigquery Spatial Temporal Data Type Transformation

In [ ]:
from google.cloud import bigquery

table_name = f'{PROJECT_ID}.geo_data.plz2_polygone'

# Create a BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Define the query to convert the column to geography
query = '''
SELECT
  plz2, ST_GEOGFROMTEXT(geometry2, make_valid => TRUE) AS polygon, qkm, einwohner
FROM
  `geo_data.plz2_polygone`
'''

# Define the configuration for the query job
job_config = bigquery.QueryJobConfig(destination=table_name,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE)

# Start the query job
job = client.query(query, job_config=job_config)

job.result()

In [ ]:
table_name = f'{PROJECT_ID}.geo_data.plz5_polygone'

# Create a BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Define the query to convert the column to geography
query = '''
SELECT
  plz5, plz2, ST_GEOGFROMTEXT(geometry5, make_valid => TRUE) AS polygon, qkm, einwohner, note
FROM
  `geo_data.plz5_polygone`
'''

# Define the configuration for the query job
job_config = bigquery.QueryJobConfig(destination=table_name,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE)

# Start the query job
job = client.query(query, job_config=job_config)

job.result()